In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.0 MB/s eta 0:00:0000:01


In [2]:
import datetime as dt
import base64
import json
import requests
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256, SHA1
from Crypto.Signature import pss

In [ ]:
class AIRuntime:
    PEM_FORMAT = "-----BEGIN PUBLIC KEY-----\n{0}\n-----END PUBLIC KEY-----\n"
    def __init__(self, access_key_id, access_key, model_id):
        self._baseEndpoint = "https://aiip.skcc.com"
        self._tokenValidMinutes = 10
        self._access_key_id = access_key_id
        self._access_key = access_key
        self._model_Id = model_id
        self._rsa_key = self.extract_rsa_key()
        self._token = self.get_token()
    def extract_rsa_key(self):
        pem_data = self.PEM_FORMAT.format(self._access_key)
        rsa_key = RSA.importKey(pem_data)
        return rsa_key
    def get_endpoint(self):
        return self._baseEndpoint + "/api/runtime/ifservice/predict"
    def get_token(self):
        _msg = str(int(dt.datetime.now().timestamp() - 1))
        _cipher = PKCS1_OAEP.new(key=self._rsa_key, hashAlgo=SHA256, mgfunc=lambda x, y: pss.MGF1(x, y, SHA1))
        _enc_str = _cipher.encrypt(str.encode(_msg))
        _b64_encoded_str = base64.b64encode(_enc_str)
        _headers = dict()
        _headers["Content-Type"] = "application/json"
        _payload = dict()
        _payload["keyId"] = self._access_key_id
        _payload["encMessage"] = bytes.decode(_b64_encoded_str)
        _payload["duration"] = self._tokenValidMinutes
        url = "https://aiip.skcc.com/api/common/backend/admin/api/keyauth"
        res = requests.post(url, headers=_headers, json=_payload)
        if res.status_code == 200:
            token = res.content.decode("utf-8")
            return token
        else:
            print("Error: ", str(res.content.decode("utf-8")))
    def predict(self, model_id, data):
        json_data = json.loads(ai_runtime.get_token())
        token = json_data['result']
        return self.predict_exec(model_id, data, token)
    def predict_exec(self, model_id, data, token):
        _headers = dict()
        _headers["Content-Type"] = "application/json"
        _headers["Api-Auth-Token"] = token
        _payload = data
        _url = self.get_endpoint() + "/" + model_id
        _res = requests.post(_url, headers=_headers, json=_payload)
        if _res.status_code == 200:
            _result = str(_res.content.decode("utf-8"))
            return _result
        else:
            print("Error: ", str(_res.content.decode("utf-8")))
apiKeyId = "A61003DE45A4"
apiKey = "MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAtgxxNY5+rCGW6Q6X/zKz2xTa4RRiLNLm6Y0FzaAqlKd6agxCIaLaFx28EFlf6aNMc/fEAYcUVs/bLKNXx3t3G88xfp/+B9MgrGkV6WRaiLY5RzySiluo5xmABVA7O/ylJrALJvOsUZOzcGruhRkfn0/WouKQUt7BTSSl6T1N3meKm/KDx3r4qdcslaNahHF2xE2Ujco5vVToxKzAqVynthMUTXrXGxTm2p05Rqz+BokoR+yDysxT4BCoraEuz16OpBQMkfaazrjZPbIbU3tqIfVlUOnQNcTBgJKakjFHsHT0qjfc6NFr9oW0LQVmKMkJxvKErkdjWNp8v7cOhIdr7QIDAQAB"
modelId = "mdl-2bc7449d-88e2-47a2-be20-456fc9334b44"
data = {
    "instances": [
        [5.1, 3.5, 1.4, 0.2]
    ],
    "labels": ["sepal_length", "sepal_width", "petal_length", "petal_width"]
}
try:
    ai_runtime: AIRuntime = AIRuntime(access_key_id=apiKeyId, access_key=apiKey, model_id=modelId)
    predict_result = ai_runtime.predict(modelId, data)
    print(predict_result)
except Exception as e:
    print(e)